In [2]:
# import pacakges
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

import pickle

# Import Data

In [3]:
X_train = pd.read_csv('data/train-test/train_nulls_dropped.csv')
X_test = pd.read_csv('data/train-test/test_nulls_dropped.csv')

In [3]:
features = ['Pitcher_name','batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features2 = ['batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features3 = ['pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']

In [4]:
X_train = X_train[features3]
X_test = X_test[features3]

In [4]:
X_train.describe(include='all')

,game_date,pitch_type,Pitcher_name,pitcher_id,batter_name,batter_id,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,inning,inning_topbot,effective_speed,release_spin_rate,release_extension,Catcher,FirstBasemen,SecondBasemen,ThirdBasemen,ShortStop,LeftField,CenterField,RightField,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,count,strike_attempt,events,description,type,bb_type,vx0,vy0,vz0,ax,ay,az,outs_when_up
count,3486421,3486421,3486421,3.486421e+06,3486421,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486420e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3486421,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3486421,3486421,3486421,3486421,894755,3486421,3486421,619883,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06,3.486421e+06
unique,1022,11,1519,NaN,2459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,4,5,14,4,35,19,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2019-09-24,FF,Justin Verlander,NaN,J.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Seam Fastball,NaN,NaN,NaN,NaN,Standard,Standard,0-0,strike,field_out,ball,S,ground_ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,5252,1279440,17300,NaN,24484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279440,NaN,NaN,NaN,NaN,2584080,3196101,900721,1603119,358345,1170277,1603175,280977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.373077e+05,NaN,5.312826e+05,8.886501e+01,-7.750183e-01,5.896235e+00,5.807362e-01,7.266888e-01,8.850156e-01,8.812011e-01,-2.075446e-01,8.000621e-01,7.577303e-03,2.268045e+00,4.981419e+04,9.742581e+04,1.603912e+05,4.997988e+00,5.089446e-01,8.837593e+01,2.207366e+03,6.023982e+00,5.159211e+05,5.133094e+05,5.435206e+05,5.390766e+05,5.565456e+05,5.329339e+05,5.465585e+05,5.317534e+05,3.905785e+01,2.897231e+00,NaN,2.281537e+00,2.326291e+00,2.281609e+00,2.326291e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.281706e+00,-1.291299e+02,-4.417043e+00,-2.500986e+00,2.642436e+01,-2.266163e+01,9.795005e-01
std,NaN,NaN,NaN,7.700898e+04,NaN,8.110774e+04,5.983508e+00,1.899442e+00,4.859079e-01,4.934387e-01,4.456594e-01,9.685690e-01,8.245336e-01,8.852363e-01,7.590396e-01,8.841680e-01,9.463187e-01,1.570542e+05,2.086829e+05,2.476457e+05,2.672705e+00,4.999201e-01,6.280629e+00,3.322019e+02,5.125216e-01,6.968181e+04,6.823719e+04,6.730462e+04,8.825135e+04,6.975141e+04,8.719408e+04,6.709532e+04,8.100020e+04,2.315099e+01,1.730426e+00,NaN,2.578869e+00,2.668753e+00,2.578902e+00,2.668753e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.035281e+00,8.663872e+00,3.051180e+00,1.075260e+01,4.080999e+00,8.846437e+00,8.174710e-01
min,NaN,NaN,NaN,1.125260e+05,NaN,1.125260e+05,3.610000e+01,-1.642000e+01,-3.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.200000e+00,-4.500000e+00,-1.054000e+01,-5.180000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,9.000000e+00,4.130000e+02,-9.000000e-01,1.502290e+05,1.200740e+05,2.171000e+05,1.213470e+05,2.171000e+05,1.368600e+05,4.000850e+05,1.163380e+05,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.202600e+01,-1.531160e+02,-1.991990e+01,-5.929009e+01,-5.243748e-01,-7.717142e+01,0.000000e+00
25%,NaN,NaN,NaN,4.754790e+05,NaN,4.602690e+05,8.480000e+01,-2.130000e+00,5.630000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.900000e-01,4.000000e-01,-5.900000e-01,1.660000e+0

In [32]:
X_train.columns[:27]

Index(['pitch_name', 'release_speed', 'release_pos_x', 'release_pos_z',
       'stand', 'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x',
       'plate_z', 'inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up'],
      dtype='object')

In [5]:
# We will pd.get_dummies 
X_train = pd.get_dummies(X_train, columns=['pitch_name'])
X_test = pd.get_dummies(X_test, columns=['pitch_name'])

In [6]:
# drop columns that don't match in each other
drop_test = [col for col in X_test.columns if col not in X_train.columns]
drop_train = [col for col in X_train.columns if col not in X_test.columns]

X_train.drop(columns=drop_train, inplace=True)
X_test.drop(columns=drop_test, inplace=True)

In [7]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [8]:
y_train = pickle.load(open('assets/y_train.pkl','rb'))
y_test = pickle.load(open('assets/y_test.pkl','rb'))

In [10]:
y_train.head()

0    strike
1    strike
2    strike
3      ball
4    strike
Name: strike_attempt, dtype: object

# Set up Pipeline/GridSearchCV

---
Unable to do GridSearch because I have been running into too many memory errors

In [11]:
# list(range(1, 101, 10))

In [12]:
# rf_params = {
#     'n_neighbors': list(range(1, 101, 10)),
#     'min_samples_split':[2,3],
#     'max_features':["auto",28],
#     'max_samples':[.3,.5,.7]
# } 

In [13]:
# rf = GridSearchCV(RandomForestClassifier(random_state=66),
#                   param_grid=rf_params,
#                   cv=5,
#                   n_jobs=8,
#                   verbose = 2)

In [14]:
# rf.fit(X_train,y_train)

In [15]:
rf = RandomForestClassifier(random_state=66, n_jobs=8)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=8, random_state=66)

In [17]:
preds = rf.predict(X_test)

In [18]:
accuracy_score(y_test, preds)

0.7092507858764223

In [21]:
df = pd.DataFrame(rf.feature_importances_, index=X_train.columns)

In [27]:
df.sort_values(0, ascending=False).head(35)

,0
plate_x,0.137504
plate_z,0.115888
vz0,0.064031
vx0,0.051252
release_pos_x,0.038365
az,0.037864
release_pos_z,0.036782
ax,0.036452
ay,0.035946
release_spin_rate,0.035828


In [28]:
pickle.dump(rf, open('assets/rf_no_pitcher_12062020.pkl','wb'))

# Random Forest model, no pitchers and batters

In [9]:
# rf_params = {
# #     'n_neighbors': list(range(1, 101, 10)),
#     'min_samples_split':[2,3],
# #     'min_samples_leaf':[1,2,3],
#     'max_features':['auto','log2'],
#     'max_samples':[.95,.99]
# } 
# rf2 = GridSearchCV(RandomForestClassifier(random_state=66),
#                   param_grid=rf_params,
#                   cv=5,
#                   n_jobs=8,
#                   verbose = 2)

In [9]:
rf2 = RandomForestClassifier(random_state=66, n_jobs=8,verbose=2)

In [10]:
rf2.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100


building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100building tree 21 of 100

building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.0min


building tree 33 of 100building tree 34 of 100

building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  6.5min finished


RandomForestClassifier(n_jobs=8, random_state=66, verbose=2)

In [11]:
preds2 = rf2.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    3.7s finished


In [12]:
accuracy_score(y_test, preds2)

0.7113739053224779

In [14]:
df2 = pd.DataFrame(rf2.feature_importances_, index=X_train.columns)

In [15]:
df2.sort_values(0, ascending=False).head(35)

,0
plate_x,0.182280
plate_z,0.147995
vz0,0.064584
vx0,0.051666
az,0.044295
release_pos_x,0.043263
release_pos_z,0.041851
ax,0.041754
release_spin_rate,0.041669
ay,0.041623
